### Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

# Apply better style
sns.set_theme(style="whitegrid")
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)
# sns.set(font='Adobe Devanagari')
sns.set_context("paper", font_scale=1, rc={"lines.linewidth": 0.5, "grid.linewidth": 0.3})


matplotlib.rcParams["mathtext.fontset"] = "cm"
matplotlib.rcParams["font.family"] = "STIXGeneral"
# Not italized latex
matplotlib.rcParams["mathtext.default"] = "regular"
matplotlib.rcParams["font.weight"] = "light"

%config InlineBackend.figure_format='png'

# Retina quality for plots
%config InlineBackend.figure_format = 'retina'

### Load data

In [ ]:
df_input = pd.read_parquet(
    "/home/HPC/cdroin/dynamic_collapse/master_study/scans/dynamic_octupoles_injection/base_collider/xtrack_0000/gen_3_normalized/input_particles.parquet"
)
df_output = pd.read_parquet(
    "/home/HPC/cdroin/dynamic_collapse/master_study/scans/dynamic_octupoles_injection/base_collider/xtrack_0000/gen_3_normalized/output_particles.parquet"
)

with open(
    "/home/HPC/cdroin/dynamic_collapse/master_study/scans/dynamic_octupoles_injection/base_collider/xtrack_0000/gen_3_normalized/observables.pkl",
    "rb",
) as f:
    dic_obs = pickle.load(f)

dic_obs["l_df_particles"][0]

### Plot initial distribution

In [ ]:
df_input_with_norm = dic_obs["l_df_particles"][0]

x_input = df_input_with_norm["x_norm"]
y_input = df_input_with_norm["y_norm"]
px_input = df_input_with_norm["px_norm"]
py_input = df_input_with_norm["py_norm"]

# Plot input particles (two subplots for x and y)
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].scatter(x_input, px_input, s=1)
ax[0].set_xlabel(r"$x$ [$\sigma$]")
ax[0].set_ylabel(r"$p_x$ [$\sigma$]")
ax[0].grid()
ax[0].set_ylim(-8, 8)
ax[0].set_xlim(-8, 8)
ax[1].scatter(y_input, py_input, s=1)
ax[1].set_xlabel(r"$y$ [$\sigma$]")
ax[1].set_ylabel(r"$p_y$ [$\sigma$]")
ax[1].grid()
ax[1].set_ylim(-8, 8)
ax[1].set_xlim(-8, 8)
plt.suptitle("Initial particles distribution")
plt.tight_layout()
plt.show()

### Plot evolution octupoles

In [ ]:
plt.plot(dic_obs["l_n_turns"], dic_obs["l_oct"])
plt.xlabel("Turns")
plt.ylabel("Octupoles intensity (A)")
#plt.xlim(1900, 2200)
#plt.ylim(0,25)
plt.grid()
plt.show()

In [ ]:
plt.scatter(range(len(dic_obs["l_n_turns"])), dic_obs["l_n_turns"])
plt.grid()
plt.show()

### Plot ellipses

In [ ]:
# Plot evolution of particles ellipses during first 50k turns (no octupoles)
cmap = matplotlib.colormaps["Spectral"]
colors = cmap(np.linspace(0, 1, len(dic_obs["l_df_particles"][0]["x_norm"])))
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
for n_turn, df in zip(dic_obs["l_n_turns"], dic_obs["l_df_particles"]):
    if n_turn > 2000:
        break
    for sigma in range(len(df["x_norm"])):
        ax[0].scatter(df["x_norm"][sigma], df["px_norm"][sigma], s=1, color=colors[sigma])
        ax[1].scatter(df["y_norm"][sigma], df["py_norm"][sigma], s=1, color=colors[sigma])
ax[0].set_xlabel(r"$x$ [$\sigma$]")
ax[0].set_ylabel(r"$p_x$ [$\sigma$]")
ax[0].grid()
ax[1].set_xlabel(r"$y$ [$\sigma$]")
ax[1].set_ylabel(r"$p_y$ [$\sigma$]")
ax[1].grid()
plt.suptitle(r"Ellipses colored by $\sigma$, no octupoles")
plt.tight_layout()
plt.show()

In [ ]:
# Plot the trajectory of a single ellipse for the whole simulation
cmap = matplotlib.colormaps["rainbow"]
max_oct = np.max(dic_obs["l_oct"]) + 50
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
for octu, df in zip(dic_obs["l_oct"], dic_obs["l_df_particles"]):
    ax[0].scatter(df["x_norm"][6], df["px_norm"][6], s=1, color=cmap(octu / max_oct))
    ax[1].scatter(df["y_norm"][6], df["py_norm"][6], s=1, color=cmap(octu / max_oct))
ax[0].set_xlabel(r"$x$ [$\sigma$]")
ax[0].set_ylabel(r"$p_x$ [$\sigma$]")
ax[0].grid()
ax[1].set_xlabel(r"$y$ [$\sigma$]")
ax[1].set_ylabel(r"$p_y$ [$\sigma$]")
ax[1].grid()

# Display colorbar
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(0, max_oct))
sm.set_array([])
cb = fig.colorbar(sm, cax=cbar_ax, orientation="vertical")
cb.outline.set_visible(False)
cb.ax.get_yaxis().labelpad = 15
cb.ax.set_ylabel("Octupoles (A)", rotation=270)
plt.suptitle("Trajectory of the $3\sigma$ ellipse for the whole simulation")
plt.show()